In [13]:
import pandas as pd
import re

In [14]:
def convert_time(time_str):
    
    """Convert time string from various formats to HH:MM:SS"""
    
    if pd.isna(time_str) or time_str == '':
        return '00:00:00'
    
    # Initialize hours, minutes, seconds
    hours = 0
    minutes = 0
    seconds = 0
    
    # Handle hours
    hour_match = re.search(r'(\d+)\s*hours?', time_str)
    if hour_match:
        hours = int(hour_match.group(1))
    
    # Handle minutes
    min_match = re.search(r'(\d+)\s*min\.?', time_str)
    if min_match:
        minutes = int(min_match.group(1))
    
    # Handle seconds
    sec_match = re.search(r'(\d+)\s*s\.?', time_str)
    if sec_match:
        seconds = int(sec_match.group(1))
    
    # Format as HH:MM:SS with leading zeros
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"



In [15]:
# Read the CSV file
df = pd.read_csv('Daily_raw.csv')

# Identify columns to convert (all except 'Name?' and 'Logged calls')
time_columns = [col for col in df.columns if col not in ['Name?', 'Logged calls']]

# Convert each time column
for col in time_columns:
    df[col] = df[col].apply(convert_time)

# Create new blank columns 
df["PENDING"] = pd.Series(dtype='int')
df["DAYS WORKED"] = pd.Series(dtype='int')

# Order columns to match the report
df = df[["Name?", "DAYS WORKED", "Ready (t)", "PENDING", "Break (t)", "Lunch (t)",
        "Submitting Lead (t)", "Bathroom (t)", "DEAD TIME (t)", "Ready:Talk Time",
         "Logged calls", "Ready:Wrap Time"]]

# Make the index starts at 1
df.index = df.index +1

# Move the index to the first column
df.reset_index(inplace=True)

# Function to normalize time strings like "00:60:00" to "01:00:00"
def normalize_time_str(t):
    if pd.isna(t) or t == '':
        return '00:00:00'
    try:
        h, m, s = [int(x) for x in t.split(':')]
        m += s // 60
        s = s % 60
        h += m // 60
        m = m % 60
        return f"{h:02d}:{m:02d}:{s:02d}"
    except Exception:
        return t  # If already a datetime or not a string, return as is

# Normalize time columns before converting to datetime
for col in df.columns[1:]:
    if col not in ['Name?', 'DAYS WORKED', 'Logged calls']:
        df[col] = df[col].apply(normalize_time_str)
        df[col] = pd.to_datetime(df[col], format='%H:%M:%S', errors='coerce').dt.time # Convert to time
        df[col] = df[col].apply(lambda x: x.strftime('%H:%M:%S') if pd.notnull(x) else '00:00:00') # Convert back to string with format HH:MM:SS

# Fill NaT values in columna DAYS WORKED with 1
df['DAYS WORKED'] = df['DAYS WORKED'].fillna(1).astype(int)

# Remove values in column PENDING
df['PENDING'] = ''

# Save the converted data (or you can just work with the dataframe)
df.to_csv('Daily_converted.csv', index=False)

# Print the first few rows of the converted dataframe
print(df.head())

# Print a message indicating completion
print("\n")
print("Conversion complete. Data saved to Daily_converted.csv")


   index       Name?  DAYS WORKED Ready (t) PENDING Break (t) Lunch (t)  \
0      1       HLizz            1  06:35:00          00:47:47  01:01:00   
1      2      GJohan            1  07:04:00          01:00:00  01:01:00   
2      3  HTVictoria            1  07:42:00          00:59:03  01:00:00   
3      4     GScooby            1  07:11:00          00:59:22  01:00:00   
4      5     HTMario            1  06:25:00          00:50:33  01:01:00   

  Submitting Lead (t) Bathroom (t) DEAD TIME (t) Ready:Talk Time  \
0            00:01:10     00:05:00      00:29:30        01:51:00   
1            00:18:51     00:00:00      00:13:46        02:33:00   
2            00:00:00     00:03:58      00:12:15        02:28:00   
3            00:23:56     00:05:02      00:11:22        02:49:00   
4            00:00:00     00:05:00      00:06:56        02:08:00   

   Logged calls Ready:Wrap Time  
0           337        00:36:49  
1           358        00:21:30  
2           415        00:17:37  
3   